In [1]:
%matplotlib inline
import sys
sys.path.insert(0,"lecture_8")
from model_runner import ModelRunner

In [2]:
model_str1 = """
     A -> B; k1*A
     B-> C; k2*B
      
     A = 50; 
     B = 0;
     C = 0;
     k1 = 0.15
     k2 = 0.2
"""
model_str2 = """
     A -> B+C; k1*A
      
     A = 50; 
     B = 0;
     C = 0;
     k1 = 0.15
"""
model_str3 = """
     A -> B; k3*A
     A-> C; k4*A
      
     A = 50; 
     B = 0;
     C = 0;
     k3 = 0.15
     k4 = 0.2
"""

simulation_time = 30
num_points = 5

In [6]:
print(model_str1)
runner = ModelRunner(model_str1, ['k1', 'k2'],
                    simulation_time, num_points)
runner.fit(count=10)


     A -> B; k1*A
     B-> C; k2*B
      
     A = 50; 
     B = 0;
     C = 0;
     k1 = 0.15
     k2 = 0.2



,mean,std
k1,0.211775,0.068438
k2,0.180459,0.080517


In [7]:
print(model_str2)
runner = ModelRunner(model_str2, ['k1'],
                    simulation_time, num_points)
runner.fit(count=5)


     A -> B+C; k1*A
      
     A = 50; 
     B = 0;
     C = 0;
     k1 = 0.15



,mean,std
k1,0.137532,0.010461


In [5]:
print(model_str3)
runner = ModelRunner(model_str2, ['k3', 'k4'],
                    simulation_time, num_points)
runner.fit(count=5)


     A -> B; k3*A
     A-> C; k4*A
      
     A = 50; 
     B = 0;
     C = 0;
     k3 = 0.15
     k4 = 0.2



,mean,std
k3,1.0,0.0
k4,1.0,0.0
